In [50]:
# Data file
path = './../data/price_data_2005-2022.csv'

# Exchanges to keep - 1. NYSE, 2. AMEX, 3. NASDAQ
filter_exchanges = True
exchanges_to_keep = [1, 3]

# Filter finacial stocks
filter_financial = False

# Fill forward zero prices
fill_forward_prices = True
fix_negative_prices = True

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
# Read in data
price_data = pd.read_csv(path)

C:\Users\amrit\AppData\Local\Temp\ipykernel_16992\2746707363.py:2: DtypeWarning: Columns (3,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  price_data = pd.read_csv(path)


In [7]:
display(price_data)

,PERMNO,date,EXCHCD,SICCD,TICKER,TRDSTAT,CUSIP,DLSTCD,DLRET,PRC,RET
0,10001,2005/01/03,3.0,4920,EWST,A,36720410,NaN,NaN,6.670,-0.033333
1,10001,2005/01/04,3.0,4920,EWST,A,36720410,NaN,NaN,6.510,-0.023988
2,10001,2005/01/05,3.0,4920,EWST,A,36720410,NaN,NaN,6.700,0.029186
3,10001,2005/01/06,3.0,4920,EWST,A,36720410,NaN,NaN,6.510,-0.028358
4,10001,2005/01/07,3.0,4920,EWST,A,36720410,NaN,NaN,6.546,0.005530
...,...,...,...,...,...,...,...,...,...,...,...
33089264,93436,2022/12/23,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,123.150,-0.017551
33089265,93436,2022/12/27,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,109.100,-0.114089
33089266,93436,2022/12/28,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,112.710,0.033089
33089267,93436,2022/12/29,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,121.820,0.080827


In [56]:
# filter out non numeric returns using regex
import re
vc = price_data['RET'].apply(lambda x: bool(re.match(r'^-?\d+(?:\.\d+)?$', str(x))))
pc = price_data[vc].value_counts()
pc

PERMNO  date        EXCHCD  SICCD   TICKER  TRDSTAT  CUSIP     DLSTCD  DLRET      PRC      RET      
10001   2017/08/03  2.0     4925    EGAS    A        36720410  233.0   0.000000   13.100   0.000000     1
82768   2022/06/24  2.0     3663    BDR     A        09369810  560.0   -0.340984  0.305    0.014637     1
82752   2007/05/21  3.0     4412.0  SNSA    A        86156510  520.0   0.007395   30.060   0.006361     1
82755   2011/05/10  3.0     3660    TLGD    A        88954210  233.0   0.001984   10.080   0.000000     1
82759   2022/03/03  3.0     7372    NUAN    A        67020Y10  233.0   0.000179   55.990   0.000179     1
                                                                                                       ..
20736   2021/11/01  1.0     1311    VEI     A        92735P10  241.0   -0.007339  17.580   0.029877     1
20737   2022/12/30  1.0     7389    ZH      A        98955N10  100.0   A          1.300    -0.057971    1
20745   2022/12/30  1.0     5074    FERG    A      

In [62]:
# PERMNO where RET == 'C'
price_data[price_data['RET'] == 'C']['PERMNO'].unique()

p = price_data[price_data['PERMNO'] == 10028]

In [18]:
if(filter_exchanges):
    # Group by permno and check if exchcd was ever in exchanges_to_keep
    exchange_filter = price_data.groupby('PERMNO')['EXCHCD'].value_counts().unstack()
    exchange_filter = exchange_filter[exchanges_to_keep].sum(axis=1) > 0
    # Filter data
    price_data = price_data[price_data['PERMNO'].isin(exchange_filter[exchange_filter].index)]

,PERMNO,date,EXCHCD,SICCD,TICKER,TRDSTAT,CUSIP,DLSTCD,DLRET,PRC,RET
27331,10051,2005/01/03,1.0,3842,HGR,A,41043F20,NaN,NaN,8.08,-0.002469
27332,10051,2005/01/04,1.0,3842,HGR,A,41043F20,NaN,NaN,8.06,-0.002475
27333,10051,2005/01/05,1.0,3842,HGR,A,41043F20,NaN,NaN,7.96,-0.012407
27334,10051,2005/01/06,1.0,3842,HGR,A,41043F20,NaN,NaN,7.99,0.003769
27335,10051,2005/01/07,1.0,3842,HGR,A,41043F20,NaN,NaN,7.79,-0.025031
...,...,...,...,...,...,...,...,...,...,...,...
33074105,93427,2022/12/23,1.0,3827.0,FN,A,G3323L10,NaN,NaN,128.69,0.002493
33074106,93427,2022/12/27,1.0,3827.0,FN,A,G3323L10,NaN,NaN,129.55,0.006683
33074107,93427,2022/12/28,1.0,3827.0,FN,A,G3323L10,NaN,NaN,126.16,-0.026167
33074108,93427,2022/12/29,1.0,3827.0,FN,A,G3323L10,NaN,NaN,129.05,0.022907


In [39]:
if(filter_financial):
    # We want to filter out all stocks that were ever in the financial sector
    # We do this by filtering out all stocks that have a SICCD that starts with 6
    in_finance = price_data.groupby('PERMNO')['SICCD'].apply(lambda x: any([str(siccd).startswith('6') for siccd in x]))
    # Filter data
    price_data = price_data[~price_data['PERMNO'].isin(in_finance[in_finance].index)]

,PERMNO,date,EXCHCD,SICCD,TICKER,TRDSTAT,CUSIP,DLSTCD,DLRET,PRC,RET
0,10001,2005/01/03,3.0,4920,EWST,A,36720410,NaN,NaN,6.670,-0.033333
1,10001,2005/01/04,3.0,4920,EWST,A,36720410,NaN,NaN,6.510,-0.023988
2,10001,2005/01/05,3.0,4920,EWST,A,36720410,NaN,NaN,6.700,0.029186
3,10001,2005/01/06,3.0,4920,EWST,A,36720410,NaN,NaN,6.510,-0.028358
4,10001,2005/01/07,3.0,4920,EWST,A,36720410,NaN,NaN,6.546,0.005530
...,...,...,...,...,...,...,...,...,...,...,...
33089264,93436,2022/12/23,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,123.150,-0.017551
33089265,93436,2022/12/27,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,109.100,-0.114089
33089266,93436,2022/12/28,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,112.710,0.033089
33089267,93436,2022/12/29,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,121.820,0.080827


In [52]:
if(fill_forward_prices):
    # Fill forward prices that are zero
    price_data['PRC'] = price_data.groupby('PERMNO')['PRC'].apply(lambda x: x.replace(0, np.nan).ffill())

,PERMNO,date,EXCHCD,SICCD,TICKER,TRDSTAT,CUSIP,DLSTCD,DLRET,PRC,RET
0,10001,2005/01/03,3.0,4920,EWST,A,36720410,NaN,NaN,6.670,-0.033333
1,10001,2005/01/04,3.0,4920,EWST,A,36720410,NaN,NaN,6.510,-0.023988
2,10001,2005/01/05,3.0,4920,EWST,A,36720410,NaN,NaN,6.700,0.029186
3,10001,2005/01/06,3.0,4920,EWST,A,36720410,NaN,NaN,6.510,-0.028358
4,10001,2005/01/07,3.0,4920,EWST,A,36720410,NaN,NaN,6.546,0.005530
...,...,...,...,...,...,...,...,...,...,...,...
33089264,93436,2022/12/23,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,123.150,-0.017551
33089265,93436,2022/12/27,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,109.100,-0.114089
33089266,93436,2022/12/28,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,112.710,0.033089
33089267,93436,2022/12/29,3.0,9999.0,TSLA,A,88160R10,NaN,NaN,121.820,0.080827


In [53]:
if(fix_negative_prices):
    # Fix negative prices
    price_data['PRC'] = abs(price_data['PRC'])